The goal of this notebook is to check if we can learn a low dimensional LSTM to solve the parity problem on very long sequences. The problem is defined as follows: for a sequence of bits, is the number of 1s in the sequence even or odd? If it is even, the output should be 0, otherwise it should be 1. Note that with sequences of length 2, this is exactly the XOR problem.

#### Imports

In [1]:
import numpy as np
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, GRU, LSTM
from tensorflow.keras import layers
from tensorflow.keras import activations

#### Dataset creation

In [2]:
n_samples = 10_000
p_test = 0.5
train_size = int(n_samples * (1 - p_test))
test_size = n_samples - train_size

arr = np.random.randint(2, size=(n_samples,))
train, test = arr[:train_size], arr[train_size:]
print(len(train), len(test))

5000 5000


#### Utility functions

In [3]:
def get_parity(a: np.array) -> int:
    return sum(a) % 2

In [4]:
def rolling_window(a: np.array, window: int) -> np.array:
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [5]:
def create_x_y(a: np.array, timesteps: int):
    # We don't have the label for the last sequence so we discard it
    x_2d = rolling_window(a, timesteps)[:-1].astype(float) 
    y = np.array([get_parity(sequence) for sequence in x_2d])
    x_3d = np.reshape(x_2d, (x_2d.shape[0], x_2d.shape[1], 1))
    return x_3d, y

In [6]:
def init_lstm():
    lstm = Sequential()
    lstm.add(LSTM(1))
    lstm.add(Dense(1, activation='sigmoid'))
    lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return lstm

In [7]:
def get_accuracy(model, x, y) -> float:
    pred = model.predict(x).round()
    pred = pred.reshape(pred.shape[0])
    n_errors = sum(pred != y)
    accuracy = 1 - (n_errors / len(y))
    return accuracy

#### Training directly with sequences of length 1000

In [8]:
timesteps = 1000
train_x, train_y = create_x_y(train, timesteps)
test_x, test_y = create_x_y(test, timesteps)

lstm = init_lstm()
lstm.fit(train_x, train_y, epochs=20, batch_size=16, verbose=0)

print('Train accuracy with sequences of length {}: {}'.format(timesteps, get_accuracy(lstm, train_x, train_y)))
print('Test accuracy with sequences of length {}: {}'.format(timesteps, get_accuracy(lstm, test_x, test_y)))

Train accuracy with sequences of length 1000: 0.499
Test accuracy with sequences of length 1000: 0.5025


Doesn't seem to work.

#### Training with sequences of length increasing from 2 to 1000.

Next we use a property of the LSTM: the same model can work with input sequences of different lengths. We will first train the LSTM with small sequences, and then increase the sequence length in training until we are able to work with sequences of length 1000.

In [9]:
timesteps_list = [2, 4, 8, 100, 1000]
test_xs, test_ys = [], []
for timesteps in timesteps_list:
    test_x, test_y = create_x_y(test, timesteps)
    test_xs.append(test_x)
    test_ys.append(test_y)

In [10]:
lstm = init_lstm()
for timesteps in timesteps_list:
    print('Training with sequences of length {}...'.format(timesteps))
    train_x, train_y = create_x_y(train, timesteps)
    test_x, test_y = create_x_y(test, timesteps)
    lstm.fit(train_x, train_y, epochs=20, batch_size=16, verbose=0)
    
    for i, timesteps_test in enumerate(timesteps_list):
        print('Test accuracy with sequences of length {}: {}'
              .format(timesteps_test, get_accuracy(lstm, test_xs[i], test_ys[i])))
    print()

Training with sequences of length 2...
Test accuracy with sequences of length 2: 1.0
Test accuracy with sequences of length 4: 0.8098478783026422
Test accuracy with sequences of length 8: 0.5392628205128205
Test accuracy with sequences of length 100: 0.5026530612244897
Test accuracy with sequences of length 1000: 0.49650000000000005

Training with sequences of length 4...
Test accuracy with sequences of length 2: 1.0
Test accuracy with sequences of length 4: 1.0
Test accuracy with sequences of length 8: 1.0
Test accuracy with sequences of length 100: 1.0
Test accuracy with sequences of length 1000: 1.0

Training with sequences of length 8...
Test accuracy with sequences of length 2: 1.0
Test accuracy with sequences of length 4: 1.0
Test accuracy with sequences of length 8: 1.0
Test accuracy with sequences of length 100: 1.0
Test accuracy with sequences of length 1000: 1.0

Training with sequences of length 100...
Test accuracy with sequences of length 2: 1.0
Test accuracy with sequence

As we can see, and as we could expect, once it works with a decent length (4), it works also for much longer lengths (1000). In this specific case, it's actually not needed to even train it with sequences of length longer than 4.